In [1]:
# !kill -9 20975

In [2]:
# !nvidia-smi

In [3]:
import os
import shutil
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="2"

In [4]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf
import cPickle as pickle

from utils import *
# from models import GCN, MLP
# from models2 import GCN_AE, GCN_SEG, MLP
import sys
sys.path.append('../')
import UtilsLJ
sys.path.append('./src/')
from model_seg import *
from utils import * 


# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

/home/mmvc/anaconda2/envs/Xiang_Li/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
batSize=1
maxStep=10000 # fixed with learningRate and learningRateDecay
learningRate=1e-3
decayStep = 30000
learningRateDecay=0.1
lrType=1 #learning rate type
lr2Period=5
adam_beta1=0.9 # check adam optimization
adam_beta2=0.99

maxGradient = 1

conWeightVar=['train/FeastNet','global_step']  #'FeastNet','global_step'# variables to be loaded
saveStep=10 # frequency to save weight
minSaveEpoch = 500
maxKeepWeights=2000 # how many records to save (for disk)
stepsContinue=-1  # from which steps continu.
#For Debug and results printing
keepProb=0.99999

dat="FeastNet-Hand-V1-wen-XYZ"
Mconv = 16

dirSave="./TFweights/Simple{}_Mconv_{}_Bs_{}_lrT_{}_LR_{}_LRdecay_{}_Dkp_{}_MaxGradient_{}".format(
    dat, Mconv, batSize, lrType, learningRate, learningRateDecay,keepProb,maxGradient)

if os.path.exists(dirSave):
    shutil.rmtree(dirSave, True)
    os.makedirs(dirSave)
else:
    os.makedirs(dirSave)
# if os.path.exists(dirSave):
# #     os.removedirs(dirSave) 
#     os.system(rm -rf dirSave)
# os.mkdir(dirSave)

#conWeightPath="./TFweights/Simple2Dto3DMGPU_NR_1_FBN_0_Ss_1024_Bs_32_lrT_1_LR_0.001_LRdecay_0.999_Dkp_0.99999"
conWeightPath=""

In [6]:
# !rm -rf dirSave
# os.rmdirs(dirSave) 

In [7]:
# Load data
f = open('PSB_Hand_wen.pkl', 'r')
all_inputs, all_adj, all_label = pickle.load(f)
f.close()
print('inputs.shape, adj.shape, label.shape', all_inputs.shape, all_adj.shape, all_label.shape)

np.random.seed(seed)
np.random.shuffle(all_inputs)
np.random.seed(seed)
np.random.shuffle(all_adj)
np.random.seed(seed)
np.random.shuffle(all_label)

all_idxs = range(len(all_label))
np.random.seed(seed)
np.random.shuffle(all_idxs)
np.save(os.path.join(dirSave, 'shuffle_idxs.npy'), all_idxs)

inputs.shape, adj.shape, label.shape (20,) (20,) (20,)


In [8]:
all_label[0].shape,all_label[1].shape, all

((17044,), (13210,), <function all>)

In [9]:
batch_size=1
nClass = 8
ARCHITECTURE=0
Mconv=32
MAX_EPOCH=10000

In [ ]:
trLen = 12

In [ ]:
# def train():

def train_one_epoch(sess, ops, train_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    loss_sum = 0.0
    total_correct = 0.0
    total_seen = 0.0
    for idx in range(0,trLen):
        # Construct feed dictionary
        input_xyz = np.expand_dims(all_inputs[idx],0)
        input_adj = np.expand_dims(all_adj[idx],0)
        input_label = np.expand_dims(all_label[idx],0)
        feed_dict = {ops['inputs_pl']: input_xyz,
                     ops['adj_pl']: input_adj,
                     ops['labels_pl']: input_label,
                     ops['is_training_pl']: is_training}
        step, _, train_pred, train_loss, train_correct = sess.run([ops['step'], ops['train_step'],
                ops['pred'], ops['loss'], ops['correct']], feed_dict=feed_dict)
        loss_sum += train_loss
        total_correct += np.sum(train_correct)
        total_seen += all_inputs[idx].shape[0]
        
#         print('train_pred, input_label', np.argmax(train_pred, axis=-1), input_label)
    
    mean_loss = loss_sum / float(trLen)
    mean_acc = total_correct / float(total_seen)
#     print('mean loss: %f' % (mean_loss))
#     print('accuracy: %f' % (mean_acc))
    return mean_loss, mean_acc

        
def eval_one_epoch(sess, ops, test_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = False
    loss_sum = 0.0
    total_correct = 0.0
    total_seen = 0.0
    for idx in range(trLen,len(all_inputs)):
        # Construct feed dictionary
        input_xyz = np.expand_dims(all_inputs[idx],0)
        input_adj = np.expand_dims(all_adj[idx],0)
        input_label = np.expand_dims(all_label[idx],0)
        feed_dict = {ops['inputs_pl']: input_xyz,
                     ops['adj_pl']: input_adj,
                     ops['labels_pl']: input_label,
                     ops['is_training_pl']: is_training}
        _, train_pred, train_loss, train_correct = sess.run([ops['train_step'],
                ops['pred'], ops['loss'], ops['correct']], feed_dict=feed_dict)
        loss_sum += train_loss
        total_correct += np.sum(train_correct)
        total_seen += all_inputs[idx].shape[0]
        
#         print('train_pred, input_label', np.argmax(train_pred, axis=-1), input_label)
        
    mean_loss = loss_sum / float(len(all_inputs)-trLen)
    mean_acc = total_correct / float(total_seen)
    print('mean loss: %f' % (mean_loss))
    print('accuracy: %f' % (mean_acc))
    return mean_loss, mean_acc

inputs_pl = tf.placeholder(tf.float32, shape=(batch_size, None, 3))
adj_pl = tf.placeholder(tf.float32, shape=(batch_size, None, 3))
labels_pl = tf.placeholder(tf.int32, shape=(batch_size,None))
is_training_pl = tf.placeholder(tf.bool, shape=(None))

# Note the global_step=batch parameter to minimize. 
# That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
batch = tf.Variable(0)

global_step = tf.Variable(1, trainable=False,name='global_step')

# Create model
with tf.variable_scope("train") as scope1:
    y_conv, end_points = get_model_xyz(inputs_pl, adj_pl, nClass, ARCHITECTURE, is_training_pl, Mconv)

# Get model and loss 

print(y_conv.shape, labels_pl.shape)
correct = tf.equal(tf.argmax(y_conv, 2), tf.to_int64(labels_pl))
accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / float(batch_size)
Loss = tf.reduce_mean(tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_pl, logits=y_conv), axis=1))

#Learning Rate****************************************************************************

lr = tf.train.exponential_decay(learningRate, global_step,
                                              decayStep, learningRateDecay, staircase=True) 
lr=tf.clip_by_value(lr, 1e-6,1)

# Optimization Algo************************************************************************
train_step = tf.train.AdamOptimizer(learning_rate=lr,
                                                beta1=adam_beta1,
                                                beta2=adam_beta2
                                               ).minimize(Loss,global_step=global_step)

saver = tf.train.Saver(max_to_keep=int(maxKeepWeights))

init_op = tf.group(tf.global_variables_initializer(),
                           tf.local_variables_initializer())

# Init variables
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())


# Continue Training************************************************************************
if len(conWeightPath)>0:
    print("Continue Training...")
    tmp_var_list={}
    if len(conWeightVar)==0:
        print("For all variables")
        globals()['conWeightVar']={''}
    else:
        print("Training variables: {}".format(conWeightVar))

    for j in conWeightVar: 
        for i in tf.global_variables():
            if i.name.startswith(j):
                tmp_var_list[i.name[:-2]] = i
                

    saver1=tf.train.Saver(tmp_var_list) 
# For Testing Dosent matther****************************************************************   


# Initialize session
sess = tf.Session()

# Add summary writers
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(os.path.join(dirSave, 'train'), sess.graph)
test_writer = tf.summary.FileWriter(os.path.join(dirSave, 'test'), sess.graph)

sess.run(init, {is_training_pl: True})


# Read Weight******************************
if len(conWeightPath)>0:
    print(conWeightPath)
    if stepsContinue==-1:            
        STEPS=sorted([int(i.split("/")[-1].split(".")[1].split("-")[-1]) for i in glob.glob(conWeightPath+"/*meta")])
        globals()['stepsContinue']=STEPS[-1]

    wtt=glob.glob(conWeightPath+"/*{}*meta".format(stepsContinue))[0][:-5]
    print("Reading Weight:{}".format(wtt))
    saver1.restore(sess,wtt)
    print('Weight is successfully updated from: {}'.format(wtt))  
#*******************************************    

ops = {'inputs_pl': inputs_pl,
       'adj_pl': adj_pl,
       'labels_pl': labels_pl,
       'pred': y_conv,
       'correct': correct,
       'loss': Loss,
       'train_step': train_step,
        'is_training_pl': is_training_pl,
        'step': global_step}

bestacc = 90
trainval_acc_list = []
for epoch in range(MAX_EPOCH):
    print('**** EPOCH %03d ****' % (epoch))
    sys.stdout.flush()

    _, cur_acc_tr = train_one_epoch(sess, ops, train_writer)
    _, cur_acc_t = eval_one_epoch(sess, ops, test_writer)
    trainval_acc_list.append([cur_acc_tr, cur_acc_t])
    # Save the variables to disk.
    if epoch>minSaveEpoch:
#         if ((epoch % saveStep == 0) & (cur_acc_t>bestacc)):
        if (cur_acc_t>bestacc):
            bestacc = cur_acc_t
            save_path = saver.save(sess, os.path.join(dirSave, "model_{}.ckpt".format(epoch)))
            print("Model saved in file: %s" % save_path)
    
    np.savetxt(os.path.join(dirSave, 'result_acc.txt'), trainval_acc_list, fmt='%d')

Tensor("train/FeastNet/FC0/Relu:0", shape=(1, ?, 16), dtype=float32)
Translation-invariant

Tensor("train/FeastNet/Conv1/Relu:0", shape=(1, ?, 32), dtype=float32)
Translation-invariant

Tensor("train/FeastNet/Conv2/Relu:0", shape=(1, ?, 64), dtype=float32)
Translation-invariant

Tensor("train/FeastNet/Conv3/Relu:0", shape=(1, ?, 64), dtype=float32)
Tensor("train/FeastNet/Lin0/Relu:0", shape=(1, ?, 256), dtype=float32)
h_cont0:
Tensor("train/FeastNet/Cont0/Relu:0", shape=(1, ?, 259), dtype=float32)
Tensor("train/FeastNet/Lin1/custom_lin/add:0", shape=(1, ?, 8), dtype=float32)
(1, ?, 8) (1, ?)


/home/mmvc/anaconda2/envs/Xiang_Li/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


**** EPOCH 000 ****
mean loss: 2.146325
accuracy: 0.472958
**** EPOCH 001 ****
mean loss: 2.484785
accuracy: 0.470558
**** EPOCH 002 ****
mean loss: 2.233383
accuracy: 0.506825
**** EPOCH 003 ****
mean loss: 1.899615
accuracy: 0.493655
**** EPOCH 004 ****
mean loss: 1.999413
accuracy: 0.496487
**** EPOCH 005 ****
mean loss: 1.520112
accuracy: 0.517365
**** EPOCH 006 ****
mean loss: 1.472615
accuracy: 0.530061
**** EPOCH 007 ****
mean loss: 1.388012
accuracy: 0.517365
**** EPOCH 008 ****
mean loss: 1.350320
accuracy: 0.529929
**** EPOCH 009 ****
